In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
from torchsummary import summary
import os

# Check if CUDA is available
if not torch.cuda.is_available():
    raise RuntimeError("CUDA is not available. Please ensure a GPU is available and CUDA is properly installed.")

device = torch.device("cuda")

# Load dataset (Fashion Minist as an example)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Grayscale hanya memiliki satu channel
])


train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

# Define CNN model
class CNN(nn.Module):
    def __init__(self, kernel_size, pooling_type="max"):
        super(CNN, self).__init__()
        if pooling_type == "max":
            self.pool = nn.MaxPool2d(2, 2)
        elif pooling_type == "avg":
            self.pool = nn.AvgPool2d(2, 2)
        
        self.conv1 = nn.Conv2d(1, 32, kernel_size=kernel_size, padding=kernel_size // 2)  # Ubah input channel dari 3 ke 1
        self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_size, padding=kernel_size // 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 256)  # Sesuaikan ukuran input ke fully connected layer
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))
        x = self.pool(nn.ReLU()(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)  # Flatten
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x


# Training function
def train_model(model, optimizer, scheduler, epochs=10, early_stopping_patience=5):
    criterion = nn.CrossEntropyLoss()
    model.to(device)
    
    train_losses = []
    test_accuracies = []
    best_accuracy = 0
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        train_loss = running_loss / len(train_loader.dataset)
        train_losses.append(train_loss)

        model.eval()
        correct = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / len(test_loader.dataset)
        test_accuracies.append(accuracy)

        scheduler.step()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {train_loss:.4f}, Accuracy: {accuracy:.2f}%")

        # Early Stopping
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered.")
            break

    return train_losses, test_accuracies



In [8]:
epochs = 5
print(f"\n=== Starting experiments for Epochs: {epochs} ===\n")

for batch_size in batch_sizes:
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    for kernel_size in kernel_sizes:
        for pooling in pooling_types:
            for opt_name, opt_func in optimizers.items():
                print(f"\nTraining model with Batch Size: {batch_size}, Kernel Size: {kernel_size}, Pooling: {pooling}, Optimizer: {opt_name}, Epochs: {epochs}")

                model = CNN(kernel_size=kernel_size, pooling_type=pooling)
                optimizer = opt_func(model.parameters(), lr=0.001)
                scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

                # Train the model
                train_losses, test_accuracies = train_model(model, optimizer, scheduler, epochs=epochs, early_stopping_patience=10)

                # Save the results for the current configuration
                results[(epochs, batch_size, kernel_size, pooling, opt_name)] = (train_losses, test_accuracies)



=== Starting experiments for Epochs: 5 ===


Training model with Batch Size: 64, Kernel Size: 3, Pooling: max, Optimizer: SGD, Epochs: 5
Epoch 1/5, Loss: 2.1977, Accuracy: 49.10%
Epoch 2/5, Loss: 1.5643, Accuracy: 69.41%
Epoch 3/5, Loss: 0.8906, Accuracy: 73.69%
Epoch 4/5, Loss: 0.6991, Accuracy: 75.81%
Epoch 5/5, Loss: 0.6298, Accuracy: 77.18%

Training model with Batch Size: 64, Kernel Size: 3, Pooling: max, Optimizer: RMSProp, Epochs: 5
Epoch 1/5, Loss: 0.4593, Accuracy: 82.01%
Epoch 2/5, Loss: 0.2727, Accuracy: 90.24%
Epoch 3/5, Loss: 0.2237, Accuracy: 90.78%
Epoch 4/5, Loss: 0.1912, Accuracy: 91.14%
Epoch 5/5, Loss: 0.1649, Accuracy: 90.88%

Training model with Batch Size: 64, Kernel Size: 3, Pooling: max, Optimizer: Adam, Epochs: 5
Epoch 1/5, Loss: 0.4192, Accuracy: 87.96%
Epoch 2/5, Loss: 0.2621, Accuracy: 90.12%
Epoch 3/5, Loss: 0.2161, Accuracy: 91.01%
Epoch 4/5, Loss: 0.1823, Accuracy: 91.50%
Epoch 5/5, Loss: 0.1539, Accuracy: 91.68%

Training model with Batch Size: 64, Kern

Epoch di jalankan Hanya 5 saja untuk mempercepat penyelesaian laporan dikarenakan bila di selesaikan lebih lanjut hingga 50, 100, 250, 512 akan memakan waktu yang sangat lama hingga berhari - hari.

Combine Epoch 5, 50, 250, 350, 500

In [ ]:
# Hyperparameter configurations
kernel_sizes = [3, 5, 7]
pooling_types = ["max", "avg"]
epochs_options = [5, 50, 100, 250, 350]
optimizers = {"SGD": optim.SGD, "RMSProp": optim.RMSprop, "Adam": optim.Adam}
batch_sizes = [64, 128, 256, 512]

# Running experiments
results = {}
for batch_size in batch_sizes:
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    for kernel_size in kernel_sizes:
        for pooling in pooling_types:
            for opt_name, opt_func in optimizers.items():
                for epochs in epochs_options:
                    print(f"\nTraining model with Batch Size: {batch_size}, Kernel Size: {kernel_size}, Pooling: {pooling}, Optimizer: {opt_name}, Epochs: {epochs}")
                    model = CNN(kernel_size=kernel_size, pooling_type=pooling)
                    optimizer = opt_func(model.parameters(), lr=0.001)
                    scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

                    train_losses, test_accuracies = train_model(model, optimizer, scheduler, epochs=epochs, early_stopping_patience=10)
                    results[(batch_size, kernel_size, pooling, opt_name, epochs)] = (train_losses, test_accuracies)

# Save results and generate output files
os.makedirs("results", exist_ok=True)
for config, (train_losses, test_accuracies) in results.items():
    batch_size, kernel_size, pooling, opt_name, epochs = config
    with open(f"results/result_bs{batch_size}_ks{kernel_size}_{pooling}_{opt_name}_ep{epochs}.txt", "w") as f:
        f.write(f"Train Losses: {train_losses}\n")
        f.write(f"Test Accuracies: {test_accuracies}\n")

print("\nAll experiments completed. Results saved to 'results/' directory.")
